# Bayesian Knowledge Tracing for Optimal Sequences
#### BKT for inferring human knowledge on a procedural task like programming

In [12]:
import numpy as np
import pandas as pd
import math
import itertools
from collections import defaultdict 
import copy
from pyBKT.generate import synthetic_data, random_model_uni
from pyBKT.fit import EM_fit
from copy import deepcopy
import matplotlib.pyplot as plt

### Importing the collected data
Our data is collected in 3 phases:
1. Pre-test (3 questions)
2. Training (6 questions)
3. Post-test (3 questions)

In [79]:
# Learning sequences (performance of each student on training data)
training_data = {
's1' : [1,0,1,1,1,0], # Blocked
's2' : [0,0,1,1,0,0], # Blocked
's3' : [0,0,1,0,1,1], # Blocked
's4' : [0,0,0,1,1,1], # Blocked
's5' : [0,1,1,1,1,1], # Blocked
's6' : [0,0,1,1,1,1], # Blocked
's7' : [1,0,1,1,1,1], # Blocked
's8' : [1,0,1,1,1,1], # Blocked
's9' : [1,0,1,0,0,1], # Blocked
's10' : [0,1,0,1,1,1], # Blocked
's11' : [1,0,0,1,1,1], # Blocked
's12' : [1,0,0,1,1,1], # Blocked
    
's13' : [1,0,1,0,1,1], # Interleaved
's14' : [0,0,0,1,1,0], # Interleaved
's15' : [1,0,1,0,0,0], # Interleaved
's16' : [1,0,0,1,1,0], # Interleaved
's17' : [0,0,0,1,1,1], # Interleaved
's18' : [1,0,0,1,1,1], # Interleaved
's19' : [0,1,1,1,0,1], # Interleaved
's20' : [0,0,0,0,1,0], # Interleaved
's21' : [1,0,0,1,0,0], # Interleaved
's22' : [0,0,0,1,1,0], # Interleaved
's23' : [0,0,1,0,1,1] # Interleaved

}

sequences = {
's1' : ['A', 'A', 'B', 'B', 'C', 'C'], # Blocked
's2' : ['B', 'B', 'A', 'A', 'C', 'C'], # Blocked
's3' : ['C', 'C', 'B', 'B', 'A', 'A'], # Blocked
's4' : ['A', 'A', 'C', 'C', 'B', 'B'], # Blocked
's5' : ['C', 'C', 'A', 'A', 'B', 'B'], # Blocked
's6' : ['B', 'B', 'C', 'C', 'A', 'A'], # Blocked
's7' : ['A', 'A', 'B', 'B', 'C', 'C'], # Blocked
's8' : ['B', 'B', 'A', 'A', 'C', 'C'], # Blocked
's9' : ['C', 'C', 'B', 'B', 'A', 'A'], # Blocked
's10' : ['A', 'A', 'C', 'C', 'B', 'B'], # Blocked
's11' : ['C', 'C', 'A', 'A', 'B', 'B'], # Blocked
's12' : ['B', 'B', 'C', 'C', 'A', 'A'], # Blocked
    
's13' : ['A', 'C', 'B', 'C', 'A', 'B'], # Interleaved
's14' : ['B', 'A', 'B', 'C', 'A', 'C'], # Interleaved
's15' : ['C', 'B', 'C', 'A', 'B', 'A'], # Interleaved
's16' : ['A', 'C', 'B', 'C', 'B', 'A'], # Interleaved
's17' : ['C', 'A', 'C', 'B', 'A', 'B'], # Interleaved
's18' : ['B', 'A', 'C', 'B', 'C', 'A'], # Interleaved
's19' : ['C', 'A', 'A', 'B', 'C', 'B'], # Interleaved
's20' : ['B', 'C', 'A', 'B', 'C', 'A'], # Interleaved
's21' : ['A', 'B', 'A', 'C', 'B', 'C'], # Interleaved
's22' : ['C', 'B', 'A', 'B', 'A', 'C'], # Interleaved
's23' : ['C', 'A', 'C', 'B', 'A', 'B'] # Interleaved
}


# Lookup for unigram/bigram sequences
lookup = {'A':1 , 'B':2, 'C':3, 'AA':4, 'BB':5, 'CC':6, 'AB':7, 'BA':8, 'BC':9, 'CB':10,
 'CA':11, 'AC':12}

# Mapping for each question from lookup table (unigrams and bigrams)
seq_lookup = [1, 4, 8, 5, 9, 6,
              2, 5, 8, 4, 12, 6,
              3, 6, 10, 5, 8, 4,
              1, 4, 12, 6, 10, 5,
              3, 6, 11, 4, 7, 5,
              2, 5, 9, 6, 11, 4, 
              1, 4, 7, 5, 9, 6,
              2, 5, 8, 4, 12, 6,
              3, 6, 10, 5, 8, 4, 
              1, 4, 12, 6, 10, 5,
              3, 6, 11, 4, 7, 5,
              2, 5, 9, 6, 11, 4,
              
              1, 12, 10, 9, 11, 7,
              2, 8, 7, 9, 11, 12, 
              3, 10, 9, 11, 7, 8,
              1, 12, 10, 9, 10, 8,
              3, 11, 12, 10, 8, 7,
              2, 8, 12, 10, 9, 11, 
              3, 11, 4, 7, 9, 10,
              2, 9, 11, 7, 9, 11, 
              1, 7, 8, 12, 10, 9,
              3, 10, 8, 7, 8, 12,
              3, 11, 12, 10, 8, 7
              ]



# Post-Test scores of each student
test_data = {
's1' : 22,
's2' : 10,
's3' : 20,
's4' : 30,
's5' : 30,
's6' : 30,
's7' : 30,
's8' : 20,
's9' : 23.3,
's10' : 30,
's11' : 30,
's12' : 26,
    
's13' : 24,
's14' : 10,
's15' : 20,
's16' : 19,
's17' : 20,
's18' : 30,
's19' : 30,
's20' : 22,
's21' : 10,
's22' : 10,
's23' : 15
}


# Pre-test scores for each student
pre_test_data = {
's1' : 0,
's2' : 0,
's3' : 10,
's4' : 20,
's5' : 20,
's6' : 0,
's7' : 0,
's8' : 0,
's9' : 10,
's10' : 0,
's11' : 0,
's12' : 0,
    
's13' : 0,
's14' : 5,
's15' : 0,
's16' : 0,
's17' : 20,
's18' : 0,
's19' : 20,
's20' : 0,
's21' : 0,
's22' : 0,
's23' : 0    
}


# empirical learning gain
learning_gain = {}
for k, v1, v2 in zip(list(pre_test_data.keys()), list(pre_test_data.values()), list(test_data.values())):
    learning_gain.update({k:v2-v1})

## Parameter Estimation in BKT
Here we are learning the parameters of the BKT Model using EM algorithm and then comparing it with the empirically obtained parameter values

In [85]:
# parameters classes
num_gs = 1 #number of guess/slip classes
num_learns = 12 #number of learning rates

# State the prior probabilities
p_T = 0.30
p_F = 0.00
p_G = 0.10
p_S = 0.30
p_L0 = 0.15

In [86]:
# Data conversion for parmater learning in BKT
lst = sum(list(training_data.values()), [])
lst_ = []
for i in lst:
    if i == 0:
        lst_.append(1)
    else:
        lst_.append(2)

In [87]:
#generate synthetic model and data.
truemodel = {}

truemodel["As"] =  np.zeros((num_learns,2,2), dtype=np.float_)
for i in range(num_learns):
    truemodel["As"][i] = np.transpose([[1-p_T, p_T], [p_F, 1-p_F]])

In [88]:
truemodel["learns"] = truemodel["As"][:,1, 0,]
truemodel["forgets"] = truemodel["As"][:,0, 1]

truemodel["pi_0"] = np.array([[1-p_L0], [p_L0]])
truemodel["prior"] = truemodel["pi_0"][1][0]

In [89]:
truemodel["guesses"] = np.full(num_gs, p_G, dtype=np.float_)
truemodel["slips"] = np.full(num_gs, p_S, dtype=np.float_)
#can optionally set learn class sequence - set randomly by synthetic_data if not included
#truemodel["resources"] = np.random.randint(1, high = num_resources, size = sum(observation_sequence_lengths))

#data!
print("generating data...")

#specifies 500 students with 100 observations for synthetic data
# observation_sequence_lengths = np.full(500, 100, dtype=np.int) 
# data = synthetic_data.synthetic_data(truemodel, observation_sequence_lengths)
data = {
    'data':np.array(lst_).reshape((1,len(lst_))),
    'starts':np.array(list(range(1, len(lst)+1, 6))),
    'lengths':np.array(len(list(range(1, len(lst)+1, 6)))*[6]),
    'resources':np.array(seq_lookup),
    'resource_names':np.array(list(range(1,13)))
}

generating data...


In [90]:
data

{'data': array([[2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2,
         2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1,
         2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2,
         2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1,
         1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2,
         2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1,
         1, 1, 2, 1, 2, 2]]),
 'starts': array([  1,   7,  13,  19,  25,  31,  37,  43,  49,  55,  61,  67,  73,
         79,  85,  91,  97, 103, 109, 115, 121, 127, 133]),
 'lengths': array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6]),
 'resources': array([ 1,  4,  8,  5,  9,  6,  2,  5,  8,  4, 12,  6,  3,  6, 10,  5,  8,
         4,  1,  4, 12,  6, 10,  5,  3,  6, 11,  4,  7,  5,  2,  5,  9,  6,
        11,  4,  1,  4,  7,  5,  9,  6,  2,  5,  8,  4, 12,  6,  3,  6, 10,
         5,  8,  4,  1,  4, 12

In [91]:
#fit models, starting with random initializations
print('fitting! each dot is a new EM initialization')

num_fit_initializations = 5
best_likelihood = float("-inf")

for i in range(num_fit_initializations):
    # include this line to randomly set initial param values
	fitmodel = random_model_uni.random_model_uni(num_learns, num_gs) 
	(fitmodel, log_likelihoods) = EM_fit.EM_fit(fitmodel, data)
	if(log_likelihoods[-1] > best_likelihood):
		best_likelihood = log_likelihoods[-1]
		best_model = fitmodel

fitting! each dot is a new EM initialization


/Users/vikramanksingh/anaconda3/lib/python3.7/site-packages/pyBKT/fit/EM_fit.py:199: RuntimeWarning: invalid value encountered in true_divide
  pair[:, 0] = (pair[:, 0] * gamma_t) / dotted
/Users/vikramanksingh/anaconda3/lib/python3.7/site-packages/pyBKT/fit/EM_fit.py:199: RuntimeWarning: invalid value encountered in true_divide
  pair[:, 0] = (pair[:, 0] * gamma_t) / dotted
/Users/vikramanksingh/anaconda3/lib/python3.7/site-packages/pyBKT/fit/EM_fit.py:200: RuntimeWarning: invalid value encountered in true_divide
  pair[:, 1] = (pair[:, 1] * gamma_t) / dotted
/Users/vikramanksingh/anaconda3/lib/python3.7/site-packages/pyBKT/fit/EM_fit.py:199: RuntimeWarning: invalid value encountered in true_divide
  pair[:, 0] = (pair[:, 0] * gamma_t) / dotted
/Users/vikramanksingh/anaconda3/lib/python3.7/site-packages/pyBKT/fit/EM_fit.py:199: RuntimeWarning: invalid value encountered in true_divide
  pair[:, 0] = (pair[:, 0] * gamma_t) / dotted
/Users/vikramanksingh/anaconda3/lib/python3.7/site-pack

/Users/vikramanksingh/anaconda3/lib/python3.7/site-packages/pyBKT/fit/EM_fit.py:199: RuntimeWarning: invalid value encountered in true_divide
  pair[:, 0] = (pair[:, 0] * gamma_t) / dotted
/Users/vikramanksingh/anaconda3/lib/python3.7/site-packages/pyBKT/fit/EM_fit.py:199: RuntimeWarning: invalid value encountered in true_divide
  pair[:, 0] = (pair[:, 0] * gamma_t) / dotted
/Users/vikramanksingh/anaconda3/lib/python3.7/site-packages/pyBKT/fit/EM_fit.py:199: RuntimeWarning: invalid value encountered in true_divide
  pair[:, 0] = (pair[:, 0] * gamma_t) / dotted
/Users/vikramanksingh/anaconda3/lib/python3.7/site-packages/pyBKT/fit/EM_fit.py:199: RuntimeWarning: invalid value encountered in true_divide
  pair[:, 0] = (pair[:, 0] * gamma_t) / dotted
/Users/vikramanksingh/anaconda3/lib/python3.7/site-packages/pyBKT/fit/EM_fit.py:200: RuntimeWarning: invalid value encountered in true_divide
  pair[:, 1] = (pair[:, 1] * gamma_t) / dotted
/Users/vikramanksingh/anaconda3/lib/python3.7/site-pack

In [92]:
# compare the fit model to the true model

print('')
print('\ttruth\tlearned')
print('prior\t%.4f\t%.4f' % (truemodel['prior'], best_model["pi_0"][1][0]))
for r in range(num_learns):
    print('learn%d\t%.4f\t%.4f' % (r+1, truemodel['As'][r, 1, 0].squeeze(), best_model['As'][r, 1, 0].squeeze()))
for r in range(num_learns):
    print('forget%d\t%.4f\t%.4f' % (r+1, truemodel['As'][r, 0, 1].squeeze(), best_model['As'][r, 0, 1].squeeze()))

for s in range(num_gs):
    print('guess%d\t%.4f\t%.4f' % (s+1, truemodel['guesses'][s], best_model['guesses'][s]))
for s in range(num_gs):
    print('slip%d\t%.4f\t%.4f' % (s+1, truemodel['slips'][s], best_model['slips'][s]))


	truth	learned
prior	0.1500	0.7663
learn1	0.3000	0.0837
learn2	0.3000	0.4847
learn3	0.3000	0.3180
learn4	0.3000	0.0015
learn5	0.3000	1.0000
learn6	0.3000	1.0000
learn7	0.3000	1.0000
learn8	0.3000	1.0000
learn9	0.3000	0.0000
learn10	0.3000	0.0000
learn11	0.3000	0.1781
learn12	0.3000	1.0000
forget1	0.0000	0.0000
forget2	0.0000	0.0000
forget3	0.0000	0.0000
forget4	0.0000	0.0000
forget5	0.0000	0.0000
forget6	0.0000	0.0000
forget7	0.0000	0.0000
forget8	0.0000	0.0000
forget9	0.0000	0.0000
forget10	0.0000	0.0000
forget11	0.0000	0.0000
forget12	0.0000	0.0000
guess1	0.1000	0.0022
slip1	0.3000	0.3916


In the learned values for the parameters, we can see that most of the learned values do fall in the range of empirical/true parameter values. We can attribute the large value of Guess to the fact that our questions were in a format (easy, hard, easy, hard) so it might lead to sequences like (0, 1, 0, 1) which is why we believe the guess probability is higher. 

# Comparing learning parameters for bigrams
Here we are comparing the learning on blocked v/s interleaved but only a pair. 

In [114]:
def compareBigrams(best_model, num_learns, lookup):
    p_learn_gram = {}
    for r in range(num_learns):
        p_learn_gram.update({list(lookup.keys())[r]:best_model['As'][r, 1, 0].squeeze()}) 
    blocked = ['AA', 'BB', 'CC']
    interleaved = ['AB', 'BA', 'BC', 'CA', 'BC', 'CB']
    b = 0
    i = 0
    for k,v in p_learn_gram.items():
        if k in blocked:
            b+= v
        elif k in interleaved:
            i+= v
    newd = {'Blocked':b/len(blocked), 'Interleaved':i/len(interleaved)}
    return p_learn_gram, newd

In [115]:
# Average learning P(L) for blocked pairs v/s interleaved pairs
a, b = compareBigrams(best_model, num_learns, lookup)

In [116]:
print("*** Learning probabilities for unigrams/bigrams ***")
a

*** Learning probabilities for unigrams/bigrams ***


{'A': 0.08371840869964726,
 'B': 0.4846537753544279,
 'C': 0.3179622507709469,
 'AA': 0.0014878007038327424,
 'BB': 1.0,
 'CC': 0.9999999918774921,
 'AB': 0.9999996606182281,
 'BA': 0.9999999999997424,
 'BC': 5.3171745141608657e-11,
 'CB': 1.0408399120381678e-05,
 'CA': 0.17806228870468765,
 'AC': 0.9999999999999999}

### On comparing all the bigrams of BLOCKED v.s INTERLEAVED, we found that Blocked proved to have a higher expected learning as compared to the Interleaved bigrams (as shown below)

In [117]:
print("Comparing the average learning P(L) for blocked pairs and interleaved pairs")
b

Comparing the average learning P(L) for blocked pairs and interleaved pairs


{'Blocked': 0.6671625975271084, 'Interleaved': 0.3630120596291584}

# Computing the Optimal Sequences
Here we use the above learned probabilities to find the best sequence. 

We frame this problem as an optimization problem where we want to find a sequences that maximizes the total learning probability but following a set of constraints. 

Objective: max_over_all_sequences(sum(P(L)))
Constraint: Sequence must have exactly 2 question of each type ('A', 'B', 'C')

In [148]:
class OptimalSequence:
    def __init__(self, dictionary):
        self.data = dictionary
        self.types = ['A', 'B', 'C']
        self.sequences = {}
    
    # Define the constraint
    def constraint(self, d):
        l = ''
        for k in d:
            l+=k
        for _ in self.types:
            if l.count(_) != 2:
                return False
        return True 
    
    # Compute all the sequences and their learnings that follow the constraint
    def findSequences(self):
        data_modified = {k:v for k,v in self.data.items() if k not in self.types}
        key_list = list(data_modified.keys())
        pairs = itertools.permutations(key_list, 3)
        for p in pairs:
            if self.constraint(p):
                res = 0
                for pi in p:
                    res+= self.data[pi]
                self.sequences.update({p:res})
        return self.sequences
    
    # Return the best sequence
    def computeOptimal(self):
        seq = self.findSequences()
        return max(seq, key=seq.get)
              
    # Return top k sequences
    def topKseq(self, num):
        seq = self.findSequences()
        s = sorted(seq, key=seq.get, reverse=True)
        return s[:num]

In [154]:
# Instantiate the optimalSequence class
optimalSequence = OptimalSequence(a)

# List down all the correct sequences
sequences = optimalSequence.findSequences()

# Return top-k sequences
topK = optimalSequence.topKseq(5)

# Return the optimal sequence
optimal = optimalSequence.computeOptimal()

In [157]:
# This is the optimal sequence
optimal

('CC', 'AB', 'BA')

In [158]:
# Here are the top-5 sequences
topK

[('CC', 'AB', 'BA'),
 ('AB', 'CC', 'BA'),
 ('CC', 'BA', 'AB'),
 ('AB', 'BA', 'CC'),
 ('BA', 'CC', 'AB')]